In [46]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LinearSVC
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.mllib.stat import Statistics
from pyspark.ml.linalg import DenseVector
from pyspark.sql import functions as F

In [47]:
import random, os
import numpy as np
from pyspark.sql import Row
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.stat import Statistics

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import abs

In [50]:
#from LC_Helper import vectorizerFunction, SmoteSampling

In [51]:
spark = SparkSession\
    .builder\
    .appName("Baseline_Model")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
    .config("spark.yarn.access.hadoopFileSystems",os.environ["STORAGE"])\
    .getOrCreate()

In [52]:
df = spark.sql("SELECT * FROM default.lc_smote_subset")

In [53]:
#Creating list of categorical and numeric features
num_cols = [item[0] for item in df.dtypes if item[1].startswith('in') or item[1].startswith('dou')]

In [54]:
df = df.dropna()

In [55]:
df = df.select(['acc_now_delinq', 'acc_open_past_24mths', 'annual_inc', 'avg_cur_bal', 'funded_amnt', 'is_default'])

In [56]:
train = df.sampleBy("is_default", fractions={0: 0.8, 1: 0.8}, seed=10)

In [57]:
test = df.subtract(train)

Creating Model Pipeline

In [59]:
#Creates a Pipeline Object including One Hot Encoding of Categorical Features  
def make_pipeline(spark_df):        
     
    for c in spark_df.columns:
        spark_df = spark_df.withColumn(c, spark_df[c].cast("float"))
    
    stages= []

    cols = ['acc_now_delinq', 'acc_open_past_24mths', 'annual_inc', 'avg_cur_bal', 'funded_amnt']
    
    spark_df = spark_df.withColumn('acc_now_delinq',abs(spark_df.acc_now_delinq))
    spark_df = spark_df.withColumn('acc_open_past_24mths',abs(spark_df.acc_open_past_24mths))
    spark_df = spark_df.withColumn('annual_inc',abs(spark_df.annual_inc))
    spark_df = spark_df.withColumn('avg_cur_bal',abs(spark_df.avg_cur_bal))
    spark_df = spark_df.withColumn('funded_amnt',abs(spark_df.funded_amnt))
    
    #Assembling mixed data type transformations:
    assembler = VectorAssembler(inputCols=cols, outputCol="features")
    stages += [assembler]    
    
    #Scaling features
    #scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
    #stages += [scaler]
    
    
    #RF Classifier
    rf = LinearSVC(featuresCol='features', labelCol='is_default')
    stages += [rf]
    
    #Creating and running the pipeline:
    pipeline = Pipeline(stages=stages)
    pipelineModel = pipeline.fit(spark_df)

    return pipelineModel

In [60]:
pipelineModel = make_pipeline(train)

In [61]:
for c in test.columns:
    test = test.withColumn(c, test[c].cast("float"))

In [62]:
df_model = pipelineModel.transform(test)

In [63]:
df_model.columns

['acc_now_delinq',
 'acc_open_past_24mths',
 'annual_inc',
 'avg_cur_bal',
 'funded_amnt',
 'is_default',
 'features',
 'rawPrediction',
 'prediction']

In [72]:
df_model.show(5)

+--------------+--------------------+-----------+-----------+-----------+----------+--------------------+--------------------+----------+
|acc_now_delinq|acc_open_past_24mths| annual_inc|avg_cur_bal|funded_amnt|is_default|            features|       rawPrediction|prediction|
+--------------+--------------------+-----------+-----------+-----------+----------+--------------------+--------------------+----------+
|   -0.07103118|         -0.44507217|-0.05017451| 0.98599064|-0.54676306|       1.0|[-0.0710311830043...|[0.97265596976254...|       0.0|
|   -0.07103118|         -0.25531146| -0.0893869|  0.1979816|-0.87687695|       0.0|[-0.0710311830043...|[0.97265595623867...|       0.0|
|   -0.07103118|          0.32007903|-0.10632743|  0.6991778|  0.5470389|       0.0|[-0.0710311830043...|[0.97265594036257...|       0.0|
|   -0.07103118|          -1.1055471|-0.21852498|-0.67667013|-0.40859836|       1.0|[-0.0710311830043...|[0.97265584515078...|       0.0|
|   -0.07103118|          -1.11839

In [66]:
input_data = df_model.rdd.map(lambda x: (x["is_default"], x["prediction"])) #, float(x["probability"][1])))

In [67]:
predictions = spark.createDataFrame(input_data, ["is_default", "prediction"])

In [68]:
predictions.dtypes

[('is_default', 'double'), ('prediction', 'double')]

#### Baseline Model

In [69]:
df_model.select("is_default", "prediction").groupby('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|  294|
+----------+-----+



In [70]:
df_model.select("is_default", "prediction").groupby('is_default').count().show()

+----------+-----+
|is_default|count|
+----------+-----+
|       1.0|  107|
|       0.0|  187|
+----------+-----+



Model Evaluation

In [73]:
evaluator = BinaryClassificationEvaluator(labelCol="is_default", rawPredictionCol="prediction")

In [74]:
auroc = evaluator.evaluate(df_model, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(df_model, {evaluator.metricName: "areaUnderPR"})

In [75]:
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))

Area under ROC Curve: 0.5000
Area under PR Curve: 0.3639


In [76]:
# Handy Spark Library 
# Creates instance of extended version of BinaryClassificationMetrics
# using a DataFrame and its probability and label columns, as the output
# from the classifier
#bcm = BinaryClassificationMetrics(df_model, scoreCol='probability', labelCol='is_default')

In [77]:
# Now we can PLOT both ROC and PR curves!
#fig, axs = plt.subplots(1, 2, figsize=(12, 4))
#bcm.plot_roc_curve(ax=axs[0])
#bcm.plot_pr_curve(ax=axs[1])
#plt.show()

In [78]:
# And get the confusion matrix for any threshold we want
y_true = df_model.select(['is_default']).collect()
y_pred = df_model.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.64      1.00      0.78       187
         1.0       0.00      0.00      0.00       107

    accuracy                           0.64       294
   macro avg       0.32      0.50      0.39       294
weighted avg       0.40      0.64      0.49       294



/home/cdsw/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cdsw/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/cdsw/.local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
#Saving pipeline to S3:
pipelineModel.write().overwrite().save(os.environ["STORAGE"]+"/pdefusco/pipeline")

In [80]:
#Saving predictions

In [ ]:
predictions\
  .write.format("parquet")\
  .mode("overwrite")\
  .saveAsTable(
    'default.LC_predictions'
)

In [ ]:
train\
  .write.format("parquet")\
  .mode("overwrite")\
  .saveAsTable(
    'default.LC_train'
)

In [ ]:
test\
  .write.format("parquet")\
  .mode("overwrite")\
  .saveAsTable(
    'default.LC_test'
)

In [100]:
spark.stop()